In [ ]:
import json
import pandas as pd
import os

In [ ]:
experiment_path = './data/results/EMNIST/alpha_inf/logs'
metrics = [
    'server_accuracy',
    'server_recall',
    'server_precision',
]

round_threshold_one = 600
round_threshold_two = 1000
accuracy_threshold_one = 0.84
accuracy_threshold_two = 0.85

In [ ]:
def read_json(path):
    with open(path) as json_file:
        return json.load(json_file)
    
def moving_average(data, length = 10):
    moving_average_data = []
    for i, x in enumerate(data):
        if i < length:
            moving_average_data.append(sum(data[0:(i+1)]) / (i + 1))
        else:
            moving_average_data.append(sum(data[(i+1-length):(i+1)]) / length)
    return moving_average_data

In [ ]:
algorithms = os.listdir(experiment_path)
data = dict()
for alg in algorithms:
    data[alg] = read_json(os.path.join(experiment_path, alg))

In [ ]:
def find_threshold(average_metric, threshold):
    metric_list = [i for (i, x) in enumerate(average_data) if x >= threshold]
    if len(metric_list) > 0:
        return metric_list[0]
    else:
        return '-'

In [ ]:
round_threshold_one_data = []
round_threshold_two_data = []
accuracy_threshold_one_data = []
accuracy_threshold_two_data = []
metric_name = []
alg_name = []
for metric in metrics:
    for alg in data:
        alg_name.append(alg)
        metric_name.append(metric)
        metric_data = data[alg][metric]
        average_data = moving_average(metric_data)
        round_threshold_one_data.append('{:.3f}'.format(average_data[round_threshold_one - 1]))
        round_threshold_two_data.append('{:.3f}'.format(average_data[round_threshold_two - 1]))
        accuracy_threshold_one_data.append(find_threshold(average_data, accuracy_threshold_one))
        accuracy_threshold_two_data.append(find_threshold(average_data, accuracy_threshold_two))

In [ ]:
r1 = 'Round {}'.format(round_threshold_one)
r2 = 'Round {}'.format(round_threshold_two)
a1 = '{}%'.format(int(accuracy_threshold_one * 100))
a2 = '{}%'.format(int(accuracy_threshold_two * 100))
table_dict = dict()
table_dict['Metric'] = metric_name
table_dict['Algorithm'] = alg_name
table_dict[r1] = round_threshold_one_data
table_dict[r2] = round_threshold_two_data
table_dict[a1] = accuracy_threshold_one_data
table_dict[a2] = accuracy_threshold_two_data

In [ ]:
df = pd.DataFrame(table_dict)

In [ ]:
print(df.to_latex(index=False)) 